In [ ]:
import cv2
import numpy as np
import os
from tqdm.contrib.concurrent import thread_map
from tqdm.auto import tqdm
from tkinter import Tk, filedialog
from multiprocessing import cpu_count
from multiprocessing.dummy import Pool as ThreadPool




# 選取資料檔案
root = Tk()
root.attributes('-topmost', True)  # 將主視窗設置為最上層
root.withdraw()

data_file_paths = filedialog.askopenfilenames(parent = root ,title='選擇資料檔案')

filename = data_file_paths


#output
width, height = 480, 480
fps = 120
savefilename = os.path.basename(filename[0])
# 輸出資料夾設定
output_dir = filedialog.askdirectory(title='選擇輸出資料夾')
ROI_num = 1
start_time = [0] #sec
duration = 30 #min
ispreview = False


# 創建VideoCapture對象
cap = cv2.VideoCapture(filename[0])
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 檢查文件是否成功打開
if not cap.isOpened():
    print('無法打開文件')

# 建立 VideoWriter 物件
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 創建窗口並設置鼠標回調函數
cv2.namedWindow('frame')
points = []
Ms = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        if len(points) == ROI_num*4:
            cv2.setMouseCallback('frame', lambda *args: None)

cv2.setMouseCallback('frame', mouse_callback)

previewed = False

# 開始讀取視頻幀
while True:
    ret, frame = cap.read()
    
    # 當讀取完所有幀，退出循環
    if not ret:
        break
        
    resized_frame = cv2.resize(frame, (1600, 900))
        
    # 在每次點擊時繪製圓圈
    for point in points:
        cv2.circle(resized_frame, point, 5, (0, 0, 255), -1)
        
    # 等待用戶選擇四個點
    if len(points) < ROI_num*4:
        cv2.imshow('frame', resized_frame)
        cv2.waitKey(1)
        continue
        
    # 計算變換矩陣
    dst_points = np.float32([[0, 0], [height, 0], [height, width], [0, width]])
    
    for i in range(ROI_num):
        src_points = np.float32((points[0+i*4],points[1+i*4],points[2+i*4],points[3+i*4]))
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        Ms.append(M)
    break
    

def process_roi(i):
    try:
        outputfull = os.path.join(output_dir, savefilename + f'_ROI_{i}.mp4')
        out = cv2.VideoWriter(outputfull, fourcc, fps, (width, height))
        cap = cv2.VideoCapture(filename[0])
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_time[i]*fps)
        now_frames = 0
        end_frame = (60*duration)*fps
        
        for _ in tqdm(range(end_frame), desc=f"Processing ROI {i}"):
            ret, frame = cap.read()
            
            if not ret or now_frames >= end_frame:
                break
                
            resized_frame = cv2.resize(frame, (1600, 900))
            warped = cv2.warpPerspective(resized_frame, Ms[i], (height, width))
            resized_frame = cv2.resize(warped, (480, 480))
            
            out.write(resized_frame)
            now_frames += 1

    except Exception as e:
        print(f"Error in thread {i}: {e}")
    finally:
        # 確保資源被釋放
        cap.release()
        out.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    thread_map(process_roi, range(ROI_num), max_workers=cpu_count())
    cv2.destroyAllWindows()

d:\miniforge3\envs\basic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import cProfile
import pstats
import av
from tqdm import tqdm
import cv2
import numpy as np
import os
import time


def process_video_frame_by_frame(args, skip_seconds=11):
    """逐幀處理影片，從第 skip_seconds 秒開始"""
    video_path, transform_matrix, output_dir, duration, fps_output, resize_dim = args

    try:
        container = av.open(video_path)
    except av.AVError as e:
        raise ValueError(f"無法打開影片: {video_path}，錯誤: {e}")

    stream = container.streams.video[0]

    original_fps = stream.average_rate if stream.average_rate is not None else 1 / stream.time_base
    frame_step = max(1, int(original_fps // fps_output))+1  # 計算跳幀數
    total_frames = int(stream.frames if stream.frames else stream.duration * original_fps)
    max_frames = total_frames if duration is None else min(total_frames, int(duration * original_fps))

    # 計算跳過的起始幀
    start_frame = int(skip_seconds * original_fps)

    # 跳過指定的幀數
    container.seek(int(start_frame / original_fps / stream.time_base))

    # 準備輸出文件
    os.makedirs(output_dir, exist_ok=True)
    base_name, ext = os.path.splitext(os.path.basename(video_path))
    out_path = os.path.join(output_dir, f"{base_name}_optimized{ext}")
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps_output, resize_dim)

    print(f"開始處理影片: {video_path}, 跳過前 {skip_seconds} 秒, 總幀數: {max_frames}, 原始 FPS: {original_fps}, 跳幀數: {frame_step}")

    with tqdm(total=max_frames // frame_step, desc=f"Processing {os.path.basename(video_path)}") as pbar:
        for i, frame in enumerate(container.decode(video=0)):
            if i < start_frame:
                continue  # 跳過起始幀
            if i >= max_frames + start_frame:
                break
            if (i - start_frame) % frame_step == 0:  # 根據跳幀數選擇幀
                # 轉換幀為 OpenCV 格式
                img = frame.to_ndarray(format="bgr24")
                warped_frame = cv2.warpPerspective(img, transform_matrix, (resize_dim[0], resize_dim[1]))  # 確保輸出尺寸匹配
                warped_frame = warped_frame.astype(np.uint8)  # 確保數據類型正確
                out.write(warped_frame)  # 寫入處理後的幀

                pbar.update(1)

    out.release()
    print(f"處理完成，輸出文件：{out_path}")



def profile_process_video(args):
    """對 process_video_frame_by_frame 函數進行性能分析"""
    profiler = cProfile.Profile()
    try:
        profiler.enable()
        process_video_frame_by_frame(args)
    finally:
        profiler.disable()
        base_name = os.path.splitext(os.path.basename(args[0]))[0]
        profiler.dump_stats(f"profile.prof")
        print(f"性能分析已保存為: profile.prof")


def select_roi_with_av(video_path, resize_dim):
    """使用 PyAV 讓用戶選擇 ROI 並返回透視變換矩陣"""
    try:
        container = av.open(video_path)
    except av.AVError as e:
        raise ValueError(f"無法打開影片: {video_path}，錯誤: {e}")

    stream = container.streams.video[0]

    # 獲取第一幀作為預覽
    for frame in container.decode(video=0):
        img = frame.to_ndarray(format="bgr24")
        break

    if img is None:
        raise ValueError(f"無法從影片 {video_path} 中讀取首幀")

    cv2.namedWindow("Select ROI")
    points = []

    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            print(f"點選點: {x, y}")
            if len(points) == 4:
                cv2.setMouseCallback("Select ROI", lambda *args: None)

    cv2.setMouseCallback("Select ROI", mouse_callback)

    while len(points) < 4:
        preview_frame = img.copy()
        for point in points:
            cv2.circle(preview_frame, point, 5, (0, 0, 255), -1)
        cv2.imshow("Select ROI", preview_frame)
        key = cv2.waitKey(1)
        if key & 0xFF == 27:  # 按下 ESC 鍵取消
            print("用戶中止操作")
            cv2.destroyAllWindows()
            return None

    cv2.destroyAllWindows()

    # 計算透視變換矩陣
    dst_points = np.float32([[0, 0], [resize_dim[1], 0], [resize_dim[1], resize_dim[0]], [0, resize_dim[0]]])
    src_points = np.float32(points)
    transform_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    print("透視變換矩陣:\n", transform_matrix)

    return transform_matrix



def select_files():
    """選擇多部影片和輸出資料夾"""
    from tkinter import Tk, filedialog
    root = Tk()
    root.attributes('-topmost', True)
    root.withdraw()

    video_paths = filedialog.askopenfilenames(title="選擇多部影片")
    output_dir = filedialog.askdirectory(title="選擇輸出資料夾")
    if not video_paths or not output_dir:
        print("未選擇影片或輸出資料夾")
        return None, None

    return video_paths, output_dir


def process_multiple_videos(video_paths, output_dir, duration=30 * 60, fps_output=30, resize_dim=(480, 480)):
    """批量處理多部影片"""
    roi_matrices = []

    for video_path in video_paths:
        print(f"正在選擇影片 ROI: {os.path.basename(video_path)}")
        roi_matrix = select_roi(video_path, resize_dim)
        if roi_matrix is None:
            print(f"跳過影片: {os.path.basename(video_path)}")
            continue
        roi_matrices.append((video_path, roi_matrix))

    args = [(video_path, matrix, output_dir, duration, fps_output, resize_dim) for video_path, matrix in roi_matrices]

    if args:
        profile_process_video(args[0])
        for arg in args[1:]:  # 順序處理其餘影片
            process_video_frame_by_frame(arg)
    print("所有影片處理完成！")


if __name__ == "__main__":
    try:
        video_files, output_folder = select_files()
        if video_files and output_folder:
            process_multiple_videos(video_files, output_folder)
    except Exception as e:
        print(f"出現錯誤: {e}")


正在選擇影片 ROI: 4H_8.MP4
開始處理影片: G:/works/Data_archive/8f_behav/grid_walking/2024_0911_IM180ul_30min/4H_8.MP4, 跳過前 11 秒, 總幀數: 7192, 原始 FPS: 120000/1001, 跳幀數: 4


Processing 4H_8.MP4: 100%|██████████| 1798/1798 [00:21<00:00, 84.67it/s] 

處理完成，輸出文件：G:/works/Data_archive/8f_behav/grid_walking/2024_0911_IM180ul_30min/cropped/bottom_view\4H_8_optimized.MP4
性能分析已保存為: profile.prof
所有影片處理完成！


In [ ]:
import cv2
import numpy as np
import os
from tqdm.contrib.concurrent import thread_map
from tqdm.auto import tqdm
from tkinter import Tk, filedialog
from multiprocessing import cpu_count
from multiprocessing.dummy import Pool as ThreadPool




# 選取資料檔案
root = Tk()
root.attributes('-topmost', True)  # 將主視窗設置為最上層
root.withdraw()

data_file_paths = filedialog.askopenfilenames(parent = root ,title='選擇資料檔案')

filename = data_file_paths


#output
width, height = 480, 480
fps = 120
savefilename = os.path.basename(filename[0])
# 輸出資料夾設定
output_dir = filedialog.askdirectory(title='選擇輸出資料夾')
ROI_num = 1
start_time = [0] #sec
duration = 30 #min
ispreview = False


# 創建VideoCapture對象
cap = cv2.VideoCapture(filename[0])
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 檢查文件是否成功打開
if not cap.isOpened():
    print('無法打開文件')

# 建立 VideoWriter 物件
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 創建窗口並設置鼠標回調函數
cv2.namedWindow('frame')
points = []
Ms = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        if len(points) == ROI_num*4:
            cv2.setMouseCallback('frame', lambda *args: None)

cv2.setMouseCallback('frame', mouse_callback)

previewed = False

# 開始讀取視頻幀
while True:
    ret, frame = cap.read()
    
    # 當讀取完所有幀，退出循環
    if not ret:
        break
        
    resized_frame = cv2.resize(frame, (1600, 900))
        
    # 在每次點擊時繪製圓圈
    for point in points:
        cv2.circle(resized_frame, point, 5, (0, 0, 255), -1)
        
    # 等待用戶選擇四個點
    if len(points) < ROI_num*4:
        cv2.imshow('frame', resized_frame)
        cv2.waitKey(1)
        continue
        
    # 計算變換矩陣
    dst_points = np.float32([[0, 0], [height, 0], [height, width], [0, width]])
    
    for i in range(ROI_num):
        src_points = np.float32((points[0+i*4],points[1+i*4],points[2+i*4],points[3+i*4]))
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        Ms.append(M)
    break
    

def process_roi(i):
    try:
        outputfull = os.path.join(output_dir, savefilename + f'_ROI_{i}.mp4')
        out = cv2.VideoWriter(outputfull, fourcc, fps, (width, height))
        cap = cv2.VideoCapture(filename[0])
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_time[i]*fps)
        now_frames = 0
        end_frame = (60*duration)*fps
        
        for _ in tqdm(range(end_frame), desc=f"Processing ROI {i}"):
            ret, frame = cap.read()
            
            if not ret or now_frames >= end_frame:
                break
                
            resized_frame = cv2.resize(frame, (1600, 900))
            warped = cv2.warpPerspective(resized_frame, Ms[i], (height, width))
            resized_frame = cv2.resize(warped, (480, 480))
            
            out.write(resized_frame)
            now_frames += 1

    except Exception as e:
        print(f"Error in thread {i}: {e}")
    finally:
        # 確保資源被釋放
        cap.release()
        out.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    thread_map(process_roi, range(ROI_num), max_workers=cpu_count())
    cv2.destroyAllWindows()

d:\miniforge3\envs\basic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
import pstats
stats = pstats.Stats("profile.prof")
stats.strip_dirs().sort_stats("time").print_stats(10)

Tue Nov 19 08:37:14 2024    profile.prof

         18308149 function calls (18308113 primitive calls) in 904.752 seconds

   Ordered by: internal time
   List reduced from 210 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  791.331  791.331  904.752  904.752 2994869184.py:11(process_video_frame_by_frame)
    53946   37.897    0.001   37.897    0.001 {warpPerspective}
    53946   20.302    0.000   20.302    0.000 {resize}
   107900   19.457    0.000   19.457    0.000 {built-in method nt.stat}
    53946    5.552    0.000    5.552    0.000 {method 'write' of 'cv2.VideoWriter' objects}
    64361    5.025    0.000    5.025    0.000 {method 'acquire' of '_thread.lock' objects}
    53946    3.934    0.000    3.934    0.000 {method 'astype' of 'numpy.ndarray' objects}
   539490    3.328    0.000    4.962    0.000 <frozen importlib._bootstrap_external>:96(_path_join)
   107898    1.622    0.000   30.910    0.000 <frozen importlib

In [15]:
import av

def get_video_info(video_path):
    try:
        container = av.open(video_path)
        print(f"檔案: {video_path}")
        print(f"總時長: {container.duration / av.time_base:.2f} 秒")
        for stream in container.streams:
            if stream.type == "video":
                print(f"視頻編碼格式: {stream.codec_context.name}")
                print(f"解析度: {stream.codec_context.width}x{stream.codec_context.height}")
                print(f"幀率: {stream.average_rate}")
            elif stream.type == "audio":
                print(f"音訊編碼格式: {stream.codec_context.name}")
                print(f"採樣率: {stream.codec_context.sample_rate} Hz")
                print(f"聲道數: {stream.codec_context.channels}")
    except av.AVError as e:
        print(f"無法打開影片: {e}")

# 測試
get_video_info(r"G:\works\Data_archive\8f_behav\grid_walking\2024_0911_IM180ul_30min\cropped\bottom_view\2A_8_optimized.MP4")


檔案: G:\works\Data_archive\8f_behav\grid_walking\2024_0911_IM180ul_30min\cropped\bottom_view\2A_8_optimized.MP4
總時長: 59.93 秒
視頻編碼格式: h264
解析度: 480x480
幀率: 30
